In [1]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
Vv_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\validation'
Lc_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet\\development'

In [60]:
data_f = pd.read_csv(os.path.join(Vv_folder,'data_features.csv'))
visual_f = pd.read_csv(os.path.join(Vv_folder,'visual_features.csv'))
dev_data_f = pd.read_csv(os.path.join(Lc_folder,'data_features.csv'))
dev_visual_f = pd.read_csv(os.path.join(Lc_folder,'visual_features.csv'))

In [14]:
data_f.head()

,user_id,sig_id,fakeness
0,352,1,True
1,352,2,True
2,352,3,True
3,352,4,True
4,352,5,True


In [15]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,0.000000,0.0,1.370572,0.000000,0.000000,0.0,0.149738,0.323547,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.186365,0.331796,0.356497,0.0
1,0.160537,0.000000,0.0,1.169396,0.239608,0.000000,0.0,0.000000,1.208937,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.141376,0.893188,0.000000,0.0
2,0.393000,0.223869,0.0,0.195907,0.000000,0.000000,0.0,0.060111,0.088993,0.194043,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.038160,0.215892,0.050661,0.0
3,0.280439,0.609959,0.0,0.392170,0.000000,0.568858,0.0,0.000000,0.246439,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.138333,1.180603,0.000000,0.000000,0.0
4,0.351282,0.000000,0.0,0.000000,0.000000,0.940088,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.398407,1.215286,0.000000,0.000000,0.0


In [16]:
dev_data_f.head()

,user_id,sig_id,fakeness
0,403,1,True
1,403,2,True
2,403,3,True
3,403,4,True
4,403,5,True


In [17]:
dev_visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.000000,1.331955,0.000000,0.562717,0.000000,0.565360,0.283220,0.000000,0.0,0.000000,...,0.000000,0.0,0.374695,0.000000,0.0,0.304021,0.0,0.0,0.000000,0.0
1,0.439652,0.577330,0.000000,0.000000,0.030665,0.000000,0.657206,0.000000,0.0,0.395038,...,1.089617,0.0,1.774396,0.456928,0.0,0.262468,0.0,0.0,0.000000,0.0
2,0.000000,0.638873,0.000000,0.000000,0.000000,0.572654,0.726881,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.284252,0.0,0.0,0.358418,0.0
3,0.713571,0.489058,0.107338,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.391948,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0
4,0.000000,0.821043,0.220040,0.000000,0.000000,0.022878,0.000000,0.209341,0.0,0.000000,...,0.263657,0.0,0.575950,0.000000,0.0,0.307809,0.0,0.0,0.000000,0.0


In [18]:
data_f_gen = data_f.loc[data_f['fakeness']==False]
data_f_fake = data_f.loc[data_f['fakeness']==True]
data_f_gen_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(1,13))]
data_f_gen_valid_12 = data_f_gen.loc[data_f_gen['sig_id'].isin(np.arange(13,25))] 
dev_data_f_gen = dev_data_f.loc[dev_data_f['fakeness']==False]

In [19]:
dev_data_f_gen.head()

,user_id,sig_id,fakeness
15909,403,1,False
15910,403,2,False
15911,403,3,False
15912,403,4,False
15913,403,5,False


In [68]:
DECISION_THRESHOLD = 0.5
scores_sum = 0
run_count = 0
gens_sum = 0
gens_r_count = 0
fakes_sum = 0
fakes_r_count = 0
randoms_sum = 0
randoms_r_count = 0
for user_id in tqdm.tqdm(data_f['user_id'].unique(), ascii=True):
    clf = SVC(C=1,gamma='scale', probability=True, kernel='rbf', random_state=1923, cache_size=200) # gens avg: 0.9433333333333331 fakes_avg: 0.8593333333333333 cumulative_avg: 0.9013333333333332
    
    y_train = pd.concat([data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True), dev_data_f.loc[dev_data_f['fakeness']==False].reset_index(drop=True)], ignore_index=True)['user_id']==user_id
    X_train = pd.concat([visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index].reset_index(drop=True),dev_visual_f.loc[dev_data_f.loc[dev_data_f['fakeness']==False].index].reset_index(drop=True)])   
    clf.fit(X_train, y_train)
    y_valid_fakes = data_f_fake.loc[(data_f_fake['user_id']==user_id)]
    y_valid_f = len(y_valid_fakes.index)*[False]
    X_valid_f = visual_f.loc[y_valid_fakes.index]
    # score = accuracy_score(y_valid_f, (clf.predict(X_valid_f)))
    score = accuracy_score(y_valid_f, (clf.predict_proba(X_valid_f)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_f,y_valid_f)
    fakes_sum += score
    fakes_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_gens = data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']==user_id]
    y_valid_g = len(y_valid_gens.index)*[True]
    X_valid_g = visual_f.loc[y_valid_gens.index]
    # score = accuracy_score(y_valid_g, (clf.predict(X_valid_g)))
    score = accuracy_score(y_valid_g, (clf.predict_proba(X_valid_g)[:,1]> DECISION_THRESHOLD))
    # score = clf.score(X_valid_g,y_valid_g)
    gens_sum += score
    gens_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    y_valid_rand_fakes = data_f_gen_valid_12.loc[choice(data_f_gen_valid_12.loc[data_f_gen_valid_12['user_id']!=user_id].index,10)]
    y_valid_rf = len(y_valid_rand_fakes.index)*[False]
    X_valid_rf = visual_f.loc[y_valid_rand_fakes.index]
    score = clf.score(X_valid_rf,y_valid_rf)
    randoms_sum += score
    randoms_r_count += 1 
    # print(score)
    scores_sum += score
    run_count += 1
    # print(scores_sum/run_count)
    print ("gens_avg:", 100*(1-(float(gens_sum)/gens_r_count)), "fakes_avg:", 100*(1-(float(fakes_sum)/fakes_r_count)), 
            "cumulative_avg:", 100*(1-((float(gens_sum)/gens_r_count)/2 + (float(fakes_sum)/fakes_r_count)/2)))



  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

gens_avg: 0.0 fakes_avg: 0.09999999999999998 cumulative_avg: 0.050000000000000044




  2%|#6                                                                                 | 1/50 [00:12<10:14, 12.54s/it]

gens_avg: 0.04166666666666674 fakes_avg: 0.09999999999999998 cumulative_avg: 0.0708333333333333




  4%|###3                                                                               | 2/50 [00:22<09:25, 11.78s/it]

KeyboardInterrupt: 

In [24]:
len(visual_f.loc[y_train.index])

12768

In [29]:
data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True)

,user_id,sig_id,fakeness
0,352,1,False
1,352,2,False
2,352,3,False
3,352,4,False
4,352,5,False
5,352,6,False
6,352,7,False
7,352,8,False
8,352,9,False
9,352,10,False


In [61]:
pd.concat([visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index].reset_index(drop=True),dev_visual_f.loc[dev_data_f.loc[dev_data_f['fakeness']==False].index].reset_index(drop=True)])

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,1.538391,0.238738,0.000000,0.422856,0.000000,0.000000,0.405460,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.625592,1.068651,0.000000,0.000000
1,0.881185,0.420114,0.000000,0.000000,0.000000,0.000000,0.000000,0.752435,0.706175,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.344274,0.380878,0.000000,0.000000
2,1.670328,0.008383,0.000000,0.009154,0.000000,0.000000,0.185835,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.006059,0.002442,0.270199,0.362483,0.000000
3,1.647597,0.073737,0.000000,0.244051,0.070543,0.243433,0.000000,0.000000,0.242237,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.530845,0.134760,0.000000,0.000000
4,1.126356,0.154314,0.000000,0.091907,0.000000,0.000000,0.000000,0.000000,0.252425,0.0,...,0.000000,0.113798,0.000000,0.000000,0.0,0.055839,0.197102,0.616486,0.182984,0.000000
5,0.953619,0.000000,0.000000,0.834280,0.000000,0.410924,0.286199,0.000000,0.000000,0.0,...,0.000000,0.077411,0.000000,0.000000,0.0,0.307589,0.681226,0.846064,0.399180,0.000000
6,1.707449,0.148022,0.000000,0.969653,0.000000,0.000000,0.128413,0.000000,0.442419,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.873955,0.979381,0.000000,0.000000
7,1.265987,0.473231,0.000000,0.736324,0.000000,0.556843,0.000000,0.000000,0.097248,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.294730,0.351320,0.264220,0.102219,0.000000
8,1.211126,0.673744,0.000000,0.142145,0.000000,0.922579,0.000000,0.000000,0.739682,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.168352,0.758563,0.738637,0.000000,0.000000
9,1.513021,0.301371,0.000000,0.784977,0.000000,0.665350,0.419067,0.000000,0.519794,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.736776,0.992232,0.220808,0.000000


In [64]:
pd.concat([data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].reset_index(drop=True), dev_data_f.loc[dev_data_f['fakeness']==False].reset_index(drop=True)], ignore_index=True)['user_id']==user_id

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
12726    False
12727    False
12728    False
12729    False
12730    False
12731    False
12732    False
12733    False
12734    False
12735    False
12736    False
12737    False
12738    False
12739    False
12740    False
12741    False
12742    False
12743    False
12744    False
12745    False
12746    False
12747    False
12748    False
12749    False
12750    False
12751    False
12752    False
12753    False
12754    False
12755    False
Name: user_id, Length: 12756, dtype: bool

In [56]:
visual_f.loc[data_f_gen_12.loc[data_f_gen_12['user_id']==user_id].index]

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
1500,1.538391,0.238738,0.000000,0.422856,0.000000,0.000000,0.405460,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.625592,1.068651,0.000000,0.0
1500,0.000000,2.635529,0.000000,0.000000,0.000000,0.461529,0.000000,0.000000,0.384594,0.0,...,0.0,0.000000,0.448651,0.000000,0.338219,0.022897,1.911816,0.761437,0.000000,0.0
1501,0.881185,0.420114,0.000000,0.000000,0.000000,0.000000,0.000000,0.752435,0.706175,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.344274,0.380878,0.000000,0.0
1501,0.881565,0.604497,0.000000,0.000000,0.000000,0.340279,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.079064,0.156117,0.000000,0.324089,1.240505,0.696404,0.000000,0.0
1502,1.670328,0.008383,0.000000,0.009154,0.000000,0.000000,0.185835,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.006059,0.002442,0.270199,0.362483,0.0
1502,0.052411,1.752031,0.000000,0.414116,0.000000,0.502442,0.000000,0.517870,0.000000,0.0,...,0.0,0.168044,0.367178,0.000000,0.000000,0.417476,1.494968,0.659777,0.000000,0.0
1503,1.647597,0.073737,0.000000,0.244051,0.070543,0.243433,0.000000,0.000000,0.242237,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.530845,0.134760,0.000000,0.0
1503,0.000000,0.943560,0.189709,0.245873,0.000000,1.290986,0.000000,0.000000,0.258106,0.0,...,0.0,0.000000,0.540725,0.000000,0.128696,0.836457,1.426069,0.341040,0.000000,0.0
1504,1.126356,0.154314,0.000000,0.091907,0.000000,0.000000,0.000000,0.000000,0.252425,0.0,...,0.0,0.113798,0.000000,0.000000,0.000000,0.055839,0.197102,0.616486,0.182984,0.0
1504,0.000000,2.758918,0.000000,0.000000,0.000000,0.667716,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.599847,0.000000,0.449869,1.476440,0.843668,0.000000,0.000000,0.0


In [59]:
visual_f.loc[1503]

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
1503,1.647597,0.073737,0.000000,0.244051,0.070543,0.243433,0.0,0.0,0.242237,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.530845,0.13476,0.0,0.0
1503,0.000000,0.943560,0.189709,0.245873,0.000000,1.290986,0.0,0.0,0.258106,0.0,...,0.0,0.0,0.540725,0.0,0.128696,0.836457,1.426069,0.34104,0.0,0.0
